In [1]:
import json
import numpy as np
import pandas as pd
from collections import Counter

from nltk import word_tokenize, sent_tokenize

from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', 200)

In [2]:
with open("../data/href2synopsis.json", encoding="utf-8") as f:
    href2synopsis = json.load(f)

    Takeaways:
        - replace \n or \r sequences with single \n
        - use language-specific tokenizer

# BOW stats
    # TODO: add actual bow stats

In [61]:
data = pd.DataFrame()

data["href"] = href2synopsis.keys()
data["synopsis"] = href2synopsis.values()

In [62]:
data = data[data["synopsis"] != ""]

In [63]:
import regex as re

data["synopsis"] = data["synopsis"].apply(
    lambda x: re.sub("\s*\n\s*", "\n", x).strip()
)

In [44]:
# data["min_ord"] = data["synopsis"].progress_apply(
#     lambda s: min([ord(char) for char in s])
# )

# data["max_ord"] = data["synopsis"].progress_apply(
#     lambda s: max([ord(char) for char in s])
# )

# data["mean_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.mean([ord(char) for char in s])
# )

# data["median_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.median([ord(char) for char in s])
# )

# data["mode_ord"] = data["synopsis"].progress_apply(
#     lambda s: Counter([ord(char) for char in s]).most_common()[0][0]
# )

  0%|          | 0/18374 [00:00<?, ?it/s]

In [64]:
# data["mode_count"] = data["synopsis"].progress_apply(
#     lambda s: Counter(s).most_common()[0][1]
# )

data["mode_ratio"] = data["synopsis"].progress_apply(
    lambda s: Counter(s).most_common()[0][1]/len(s)
)

  0%|          | 0/18374 [00:00<?, ?it/s]

In [12]:
data["char_total"] = data["synopsis"].progress_apply(len)
data["token_total"] = data["synopsis"].progress_apply(
    lambda x: len(word_tokenize(x))
)
data["sent_total"] = data["synopsis"].progress_apply(
    lambda x: len([sent for line in x.split("\n") for sent in  sent_tokenize(line) if line.strip()])
)

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

In [27]:
# data["lines_total"] = data["synopsis"].progress_apply(lambda x: len(x.split("\n")))

  0%|          | 0/19393 [00:00<?, ?it/s]

In [5]:
data["token_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in word_tokenize(x)]) if x.strip() else 0
)

data["sent_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in sent_tokenize(x)]) if x.strip() else 0
)

data["line_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in x.split("\n")]) if x.strip() else 0
)

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

In [16]:
data["sent_char_ratio"] = [t.sent_total/t.char_total for t in data.itertuples()]
data["sent_token_ratio"] = [t.sent_total/t.token_total for t in data.itertuples()]
data["token_char_ratio"] = [t.token_total/t.char_total for t in data.itertuples()]
data["token_sent_ratio"] = [t.token_total/t.sent_total for t in data.itertuples()]
data["char_sent_ratio"] = [t.char_total/t.sent_total for t in data.itertuples()]
data["char_token_ratio"] = [t.char_total/t.token_total for t in data.itertuples()]

In [13]:
data.describe().astype(int)

,char_total,token_total,sent_total
count,18374,18374,18374
mean,519,105,5
std,348,70,3
min,1,1,1
25%,262,53,3
50%,454,92,5
75%,700,141,7
max,5799,1201,79


In [66]:
stats = data.drop(columns=["href", "synopsis"]) #.drop(columns=["line_max_char_len"])

# Outlier detection

[The svm.OneClassSVM is known to be sensitive to outliers and thus does not perform very well for outlier detection](https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection)

In [68]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [69]:
clf = LocalOutlierFactor(n_neighbors=20)
preds = clf.fit_predict(stats)

data["pred"] = preds
data["negative_outlier_factor_"] = clf.negative_outlier_factor_

In [70]:
# data[data["pred"] == -1]["synopsis"].sample(10).to_list()

In [71]:
data[data["pred"] == -1].sort_values(by="negative_outlier_factor_"
                                    )[['href', 'synopsis', 'pred', 'negative_outlier_factor_']].head(10)

,href,synopsis,pred,negative_outlier_factor_
6519,/728809-avataro-sentai-donbrothers-meets-senpaiger,"What's up with all those transformations, DON Momotaro!? Avatar Changes into veteran heroes! How many of them you DON't know?\n(Source: powerrangers.fandom.com)",-1,-6.032770e+06
4895,/736283-xi-juan-chang-an-cheng,"In a dilapidated mansion under heavy debts, there are two heaven-sent marriages across social classes, three young women and four brothers.\n(Source: ChineseDrama.info)",-1,-5.861415e+06
11337,/728943-astrophile-special,A behind-the-scenes look at the creation of Astrophile with interviews from the cast and creator.\n(Source: MyDramaList),-1,-5.555888e+06
7637,/78585-before-i-love-you-phu-x-tawan-uncut,This UnCut scene tells the feelings of both before starting a relationship.\n(Source: heyhotshot at MyDramaList),-1,-4.615862e+06
17696,/28522-amo,"Despite the Philippine government's crackdown on narcotics, high schooler Joseph expands his drug business while his cop uncle profits from corruption.\n(Source: Netflix)",-1,-4.090938e+06
3199,/75551-light-the-night,"In the red-light district of 1980s Taipei, women at a popular Japanese nightclub navigate jealousy, heartbreak, friendship, love, and betrayal.\n(Source: Netflix)",-1,-3.268206e+06
8322,/695971-sf9-center,This show is about Inseong and Jaeyoon granting the members' wishes.\n(Source: DeobiMars at MyDramaList),-1,-2.009253e+06
18615,/740141-glamorous-chinese-ethnic-idol-season-2,"In 2014, the second season of ""Glamorous Chinese Ethnic Idol"" continued. Among the seven ethnic minorities of Tujia, Miao, Dai, Yao, Dong, Daur and Kazakh, after national registration, competition...",-1,-1.618614e+06
3739,/3587-seija-no-koushin,"Exploring the perception and treatment of mentally-challenged youth and their similarities to juvenile delinquents in Japanese society, this drama series follows the trials and tribulations of a g...",-1,-1.608829e+06
16614,/7774-young-style,An up-to-the-minute look at the pressure-cooker environment of Beijing high schoolers prepping for college entry.,-1,-1.489695e+06
